In [1]:
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
import h5py
import scipy
import soccerdata as sd

%matplotlib inline
%load_ext autoreload
%autoreload 2

[11/07/22 11:24:16] INFO     No custom team name replacements found. You can configure these in       ]8;id=838873;file:///Users/selbl/opt/anaconda3/lib/python3.8/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=448855;file:///Users/selbl/opt/anaconda3/lib/python3.8/site-packages/soccerdata/_config.py#85\85]8;;\
                             /Users/selbl/soccerdata/config/teamname_replacements.json.                            

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=947003;file:///Users/selbl/opt/anaconda3/lib/python3.8/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=702584;file:///Users/selbl/opt/anaconda3/lib/python3.8/site-packages/soccerdata/_config.py#155\155]8;;\
                             /Users/selbl/soccerdata/config/league_dict.json.                                      

In [2]:
#Find path of each data files
eng_path = '/Users/selbl/Desktop/PhD/Second Year/First Quarter/CS 230/Project/Data/ENG-League-2016-2020.csv'
esp_path = '/Users/selbl/Desktop/PhD/Second Year/First Quarter/CS 230/Project/Data/ESP-League-2016-2020.csv'
fra_path = '/Users/selbl/Desktop/PhD/Second Year/First Quarter/CS 230/Project/Data/FRA-League-2016-2020.csv'
ger_path = '/Users/selbl/Desktop/PhD/Second Year/First Quarter/CS 230/Project/Data/GER-League-2016-2020.csv'
ita_path = '/Users/selbl/Desktop/PhD/Second Year/First Quarter/CS 230/Project/Data/ITA-League-2016-2020.csv'
test_path =  '/Users/selbl/Desktop/PhD/Second Year/First Quarter/CS 230/Project/Data/ENG-League-2020-2021.csv'
#Read each dataframe
df_eng = pd.read_csv(eng_path)
df_esp = pd.read_csv(esp_path)
df_fra = pd.read_csv(fra_path)
df_ger = pd.read_csv(ger_path)
df_ita = pd.read_csv(ita_path)
#Combine
df = [df_eng, df_esp, df_fra,df_ger,df_ita]
df_train = pd.concat(df)
#Get test data
df_test = pd.read_csv(test_path)

In [3]:
#Prepare databases as arrays
X_Train = np.array(df_train[['Home_Elo','Away_Elo']]).T
X_Test = np.array(df_test[['Home_Elo','Away_Elo']]).T

In [4]:
#Pre-Process Y
Y_Train = np.array(df_train['Result'])
Y_Train.reshape(1,len(Y_Train))
Y_Train = np.where(Y_Train == 'W',1,np.where(Y_Train=='L',0,2))
#Same for test
Y_Test = np.array(df_test['Result'])
Y_Test.reshape(1,len(Y_Test))
Y_Test = np.where(Y_Test == 'W',1,np.where(Y_Test=='L',0,2))

In [5]:
#Define softmax
def softmax(z):
    
    # z--> linear part.
    
    # subtracting the max of z for numerical stability.
    exp = np.exp(z - np.max(z))
    
    # Calculating softmax for all examples.
    for i in range(len(z)):
        exp[i] /= np.sum(exp[i])
        
    return exp

In [6]:
def one_hot(y, c):
    
    # y--> label/ground truth.
    # c--> Number of classes.
    
    # A zero matrix of size (m, c)
    y_hot = np.zeros((len(y), c))
    
    # Putting 1 for column where the label is,
    # Using multidimensional indexing.
    y_hot[np.arange(len(y)), y] = 1
    
    return y_hot

In [7]:
def leakyRelu(z):
    ret = max([z,0.001*z])
    return ret

In [8]:
#Define fit
def fit(X, y, lr, c, epochs):
    
    # X --> Input.
    # y --> true/target value.
    # lr --> Learning rate.
    # c --> Number of classes.
    # epochs --> Number of iterations.
    
        
    # m-> number of training examples
    # n-> number of features 
    m, n = X.shape
    
    #Normalize
    #This is because for some reason things blow up if not
    X = (X - np.mean(X))/(np.std(X))
    
    # Initializing weights and bias randomly.
    w = np.random.randn(n, c)*0.001
    b = np.random.randn(c)*0.001
    # Empty list to store losses.
    losses = []
    
    # Training loop.
    for epoch in range(epochs):
        
        # Calculating hypothesis/prediction.
        z = X@w + b
        y_hat = softmax(z)
        
        # One-hot encoding y.
        y_hot = one_hot(y, c)
        #y_hot = y
        
        # Calculating the gradient of loss w.r.t w and b.
        w_grad = (1/m)*np.dot(X.T, (y_hat - y_hot)) 
        b_grad = (1/m)*np.sum(y_hat - y_hot)
        
        # Updating the parameters.
        w = w - lr*w_grad
        b = b - lr*b_grad
        
        # Calculating loss and appending it in the list.
        loss = -np.mean(np.log(y_hat[np.arange(len(y)), y]))
        #loss = -np.mean(np.log(y_hat))
        #loss = -np.mean(np.multiply(y,y_hat))
        losses.append(loss)
        # Printing out the loss at every 100th iteration.
        if epoch%100==0:
            print('Epoch {epoch}==> Loss = {loss}'
                  .format(epoch=epoch, loss=loss))
    return w, b, losses

In [9]:
#Fit
w, b, l = fit(X_Train.T, Y_Train, lr=0.8, c=3, epochs=5000)

Epoch 0==> Loss = 1.0986077846319704
Epoch 100==> Loss = 0.9992840398253081
Epoch 200==> Loss = 0.9992840398253038
Epoch 300==> Loss = 0.9992840398253037
Epoch 400==> Loss = 0.9992840398253037
Epoch 500==> Loss = 0.9992840398253037
Epoch 600==> Loss = 0.9992840398253037
Epoch 700==> Loss = 0.9992840398253037
Epoch 800==> Loss = 0.9992840398253037
Epoch 900==> Loss = 0.9992840398253037
Epoch 1000==> Loss = 0.9992840398253037
Epoch 1100==> Loss = 0.9992840398253037
Epoch 1200==> Loss = 0.9992840398253037
Epoch 1300==> Loss = 0.9992840398253037
Epoch 1400==> Loss = 0.9992840398253037
Epoch 1500==> Loss = 0.9992840398253037
Epoch 1600==> Loss = 0.9992840398253037
Epoch 1700==> Loss = 0.9992840398253037
Epoch 1800==> Loss = 0.9992840398253037
Epoch 1900==> Loss = 0.9992840398253037
Epoch 2000==> Loss = 0.9992840398253037
Epoch 2100==> Loss = 0.9992840398253037
Epoch 2200==> Loss = 0.9992840398253037
Epoch 2300==> Loss = 0.9992840398253037
Epoch 2400==> Loss = 0.9992840398253037
Epoch 2500==

In [10]:
#Now, we set the criteria to evaluate model
def predict(X, w, b):
    
    # X --> Input.
    # w --> weights.
    # b --> bias.
    
    # Predicting
    z = X@w + b
    y_hat = softmax(z)
    
    # Returning the class with highest probability.
    return np.argmax(y_hat, axis=1)

In [11]:
def accuracy(y, y_hat):
    return np.sum(y==y_hat)/len(y)

In [12]:
#Measurements
# Accuracy for training set.
train_preds = predict(X_Train.T, w, b)
acc_train = accuracy(Y_Train, train_preds)
print('The accuracy for the train set is: ' + str(acc_train))
# Accuracy for test set.
# Flattening and normalizing.
test_preds = predict(X_Test.T, w, b)
acc_test = accuracy(Y_Test, test_preds)
print('The accuracy for the test set is: ' + str(acc_test))

The accuracy for the train set is: 0.46038987971795936
The accuracy for the test set is: 0.5289473684210526


In [14]:
#Compare with the other values from the value in SD
#First import
five38 = sd.FiveThirtyEight('ENG-Premier League', 2020)
games = five38.read_games()

[11/07/22 11:40:25] INFO     Saving cached data to /Users/selbl/soccerdata/data/FiveThirtyEight       ]8;id=724245;file:///Users/selbl/opt/anaconda3/lib/python3.8/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=31670;file:///Users/selbl/opt/anaconda3/lib/python3.8/site-packages/soccerdata/_common.py#89\89]8;;\

Index(['date', 'status', 'leg', 'home_team', 'away_team', 'home_id', 'away_id',
       'home_code', 'away_code', 'prob_home', 'prob_away', 'prob_tie', 'round',
       'matchday', 'score_home', 'score_away', 'adj_score_home',
       'adj_score_away', 'chances_home', 'chances_away', 'moves_home',
       'moves_away', 'aggregate_winner', 'shootout_winner'],
      dtype='object')

In [23]:
#Then run the codes
#This first function fetches the result by comparing the scores
def Result(df):
    #Create list for results
    l_res = []
    #Grab length
    length = len(df)
    for i in range(length):
        s_h = df['score_home'].iloc[i]
        s_a = df['score_away'].iloc[i]
        if (s_h > s_a):
            res = 'W'
        elif (s_a > s_h):
            res = 'L'
        else:
            res = 'D'
        l_res.append(res)
    return l_res

#Apply it to our dataset
res_true = Result(games)

In [24]:
#Now get the prediction
def DBPredicts(df):
    #Create list for results
    l_res = []
    #Grab length
    length = len(df)
    for i in range(length):
        p_h = df['prob_home'].iloc[i]
        p_a = df['prob_away'].iloc[i]
        p_t = df['prob_tie'].iloc[i]
        #assemble in list
        l_p = [p_h,p_a,p_t]
        #get max
        m_p = max(l_p)
        if m_p == p_h:
            res = 'W'
        elif m_p == p_a:
            res = 'L'
        else:
            res = 'D'
        l_res.append(res)
    return l_res
#res_preds
res_preds = DBPredicts(games)

In [28]:
#Compare
prob_db = sum(a == b for a,b in zip(res_preds, res_true))/len(res_true)
print('In the database, the predictions are ' + str(prob_db*100) + '% accurate')

In the database, the predictions are 52.10526315789473% accurate
